# 练一个基于扩散模型（Diffusion Model）的运动生成模型

1. 设置环境
2. 数据加载
3. 模型配置和实例化
4. 训练准备
5. 模型训练/模型加载和采样
6. 结果保存

### 1. 环境设置

In [1]:
import sys
import os

# 获取上级目录的绝对路径
path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)


### 2. 数据加载

In [2]:
from diffusion.data_loaders.motion_dataset_v2 import MotionDataset

# 加载指定的运动数据文件，设置shuffle=True以随机打乱数据
dataset = MotionDataset("data/motions/humanoid3d_cartwheel.txt", shuffle=True)

# 输出数据集的长度、第一个样本和其轨迹的形状
len(dataset), dataset[0], dataset[0].trajectories.shape

# 160*69

Tmp angle [0.0, 0.0, 0.85536, 0.9966429999999997, -0.0070009999999999795, 0.08157, 0.0005729999999999971, 0.042303731260289315, -0.056088768155961526, -0.01172717680484046, -0.014103614145860938, 0.2358842735659614, 0.37124889801787253, -0.6111023347690596, -0.09268300376873025, -0.09541896434572254, 0.585361, 0.1699928747321186, 0.08652758875118252, 0.354108626550405, 0.160215, -0.2285399691330798, -0.39445967594673703, -0.1178224382194308, -0.369571, 0.20448116583595066, -0.12115992907931128, 0.07892319943485762, 0.3736623102073797, -0.010008232584494297, 0.30603690929303384, -0.364281, -0.13425257761871864, -0.004787718949892447, 0.0010873114649849894] 35
[-0.23938   2.078199 -0.008457]
[-0.23938   2.078199]


(160,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8554,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0037, -0.0062,  0.8563,  ..., -0.2435, -1.1484, -0.8920],
         [ 0.0064, -0.0122,  0.8575,  ..., -0.0278, -1.2998, -0.9550],
         ...,
         [-0.2466,  2.0540,  0.8465,  ..., -0.8048,  0.5575,  1.2816],
         [-0.2435,  2.0658,  0.8467,  ..., -0.7320,  0.5210,  1.2596],
         [-0.2394,  2.0782,  0.8469,  ..., -0.5900,  0.5328,  1.0961]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.5536e-01,  9.9664e-01, -7.0010e-03,
          8.1570e-02,  5.7300e-04,  4.2304e-02, -5.6089e-02, -1.1727e-02,
         -1.4104e-02,  2.3588e-01,  3.7125e-01, -6.1110e-01, -9.2683e-02,
         -9.5419e-02,  5.8536e-01,  1.6999e-01,  8.6528e-02,  3.5411e-01,
          1.6022e-01, -2.2854e-01, -3.9446e-01, -1.1782e-01, -3.6957e-01,
          2.0448e-01, -1.2116e-01,  7.8923e-02,  3.7366e-01, -1.0008e-02,
          3.0604e-01, -3.6428e-01, -1.3425e-01, -4.7877e-03,  1.0873e-03,
 

### 3. 数据保存函数

In [3]:
import numpy as np

# 目标路径 DeepMimic_diffusion_mujoco/diffusion/logs/0-test

# 获取当前脚本所在的目录作为项目根目录
project_root = os.getcwd()  # 获取当前Notebook的工作目录

# 定义日志和保存路径
log_root = os.path.join(project_root)
savepath = os.path.join(project_root, "logs", "0-test")

# 如果路径不存在则创建
if not os.path.exists(savepath):
    os.makedirs(savepath)

def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    
    # 提取指定样本的轨迹数据，并对部分维度进行处理
    pos_data = dataset[80].trajectories[:, :35]
    pos_data[:, :2] = -pos_data[:, :2]
    pos_data[:, 4:8] = -pos_data[:, 4:8]
    
    # 将数据转换为NumPy数组并保存
    pos_data = pos_data.cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")

# 调用函数保存运动数据
save_motions(None, f"{savepath}", filename="test2.npy")


Motion saved as test2.npy


### 4. 模型配置和实例化

In [4]:
# 初步

import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

# 实验名称和保存路径
exp_name = "test-cartwheel-shuffled-128"
savepath = os.path.join(project_root, "logs", exp_name)

# 检查并创建必要的目录
if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

# 检查可用的设备（CUDA、MPS或CPU）
if torch.cuda.is_available():  # 如果有NVIDIA的GPU可用
    device = "cuda"
else:
    device = "cpu"

In [5]:
# 初始化模型

from diffusion.diffuser.models.temporal_v2 import TemporalUnet

# 获取数据的时间步长和维度信息
horizon = dataset[0].trajectories.shape[0]  # 获取数据的时间步数，即每个样本的时间序列长度
pos_dim = 35                                # 定义位置数据的维度数，用于模型输入或条件输入的大小
vel_dim = 34                                # 定义速度数据的维度数，可能用于扩散模型中的动作维度
transition_dim = dataset[0].trajectories.shape[1]  # 获取输入数据的总维度，包含位置、速度等

# 创建模型配置
model_config = dConfig(
    TemporalUnet,                           # 传入模型类 TemporalUnet，指定我们将使用的模型类型
    savepath=(savepath, 'model_config.pkl'), # 模型配置保存路径，用于记录和加载模型设置
    horizon=horizon,                         # 模型的时间步数（序列长度）
    transition_dim=transition_dim,           # 模型的输入维度，表示每个时间步的数据特征数
    cond_dim=transition_dim,                 # 条件输入的维度，和 transition_dim 相同，用于条件生成任务
    device=device,                           # 设置设备（CPU 或 GPU）用于模型的计算
)


# 实例化模型
model = model_config()



[utils/config ] Config: <class 'diffusion.diffuser.models.temporal_v2.TemporalUnet'>
    cond_dim: 69
    horizon: 160
    transition_dim: 69

[ utils/config ] Saved config to: /Users/xieyongyan/Desktop/third year 项目/Third_year/DeepMimic_diffusion_mujoco/diffusion/logs/test-cartwheel-shuffled-128/model_config.pkl

[ models/temporal ] Channel dimensions: [(69, 128), (128, 256), (256, 512), (512, 1024)]
[(69, 128), (128, 256), (256, 512), (512, 1024)]


In [6]:
# 初始化模型设置 参数

# 导入 GaussianDiffusion 类，这是一个用于扩散模型的自定义类
# GaussianDiffusion 是一种生成模型，通过将数据逐步噪声化、再通过学习去噪过程来生成新数据
from diffusion.diffuser.models.diffusion_v2 import GaussianDiffusion

# -------------------------------
# 定义扩散模型的参数
# -------------------------------

# 设置扩散过程的时间步数
# n_timesteps 决定了扩散过程的时间长度，即模型在噪声化和去噪过程中分成的步数
n_timesteps = 1000

# 定义损失函数类型
# 'l2' 表示均方误差损失（L2损失），它适合回归任务，因为它会惩罚偏离预测值的较大误差
loss_type = 'l2'

# 是否对去噪后的结果进行裁剪
# 若为 True，则去噪后的数据会被裁剪在一个合理的范围内，防止数据值异常偏离
# 这里设为 False，不进行裁剪
clip_denoised = False

# 是否预测噪声 epsilon
# 若为 True，模型直接预测加入噪声的数据与原数据的差异（即噪声项），适合一些特殊去噪任务
# 这里设为 False，意味着模型不直接预测噪声
predict_epsilon = False

# 设置动作（或速度）的损失权重
# 在损失计算中会考虑动作部分的权重，如果这个值较大，动作部分的误差在总损失中占比更高
# 这里设为 5 表示动作误差的权重较大
action_weight = 5

# 自定义损失权重，默认设为 None
# 如果设置为特定权重，则损失的各部分会根据这些权重加权后求和
# 这里没有自定义权重，因此设为 None
loss_weights = None

# 设置损失的折扣因子
# 如果有多个时间步，可以设置折扣因子使靠近未来的误差在损失中占比更高
# 这里设为 1，即不对误差进行折扣
loss_discount = 1

# -------------------------------
# 创建扩散模型的配置
# -------------------------------

# 使用 dConfig 创建配置对象，为 GaussianDiffusion 设定必要的参数
diffusion_config = dConfig(
    GaussianDiffusion,                 # 模型类，指定使用 GaussianDiffusion
    savepath=(savepath, "diffusion_config.pkl"),  # 配置文件保存路径，便于后续加载或检查配置
    horizon=horizon,                   # 时间序列长度，即每个样本的时间步数
    observation_dim=pos_dim,           # 观测数据的维度（位置数据）
    action_dim=vel_dim,                # 动作数据的维度（速度数据）
    n_timesteps=n_timesteps,           # 扩散过程的时间步数，控制噪声化的细化程度
    loss_type=loss_type,               # 使用的损失函数类型
    clip_denoised=clip_denoised,       # 是否对去噪后的输出进行裁剪
    predict_epsilon=predict_epsilon,   # 是否直接预测噪声
    action_weight=action_weight,       # 动作误差的损失权重
    loss_weights=loss_weights,         # 自定义损失权重，若为 None 则不使用
    loss_discount=loss_discount,       # 损失的折扣因子，若为 1 则不使用折扣
    device=device                      # 运行设备，指定在 CPU 或 GPU 上进行计算
)

# -------------------------------
# 实例化扩散模型
# -------------------------------

# 创建扩散模型实例
# 传入上一步定义的 TemporalUnet 模型作为基础神经网络，负责去噪的核心计算
diffusion = diffusion_config(model)



[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion_v2.GaussianDiffusion'>
    action_dim: 34
    action_weight: 5
    clip_denoised: False
    horizon: 160
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 1000
    observation_dim: 35
    predict_epsilon: False

[ utils/config ] Saved config to: /Users/xieyongyan/Desktop/third year 项目/Third_year/DeepMimic_diffusion_mujoco/diffusion/logs/test-cartwheel-shuffled-128/diffusion_config.pkl



### 5. 训练模型 和 训练参数设置

In [7]:
# -------------------------------
# 训练参数设置
# -------------------------------

# 学习率
# 控制模型参数更新的步长。在梯度下降过程中，学习率越高，参数调整越大，但可能导致不稳定
# 设置较小的学习率（2e-4）可以在保持训练稳定的同时有效地收敛
learning_rate = 2e-4

# 梯度累积的步数
# 指定每隔多少个步数更新一次模型的参数。通过在多个步数上累积梯度后再更新，模拟更大的批次大小
# 这样可以在内存有限的情况下，依然达到大批次训练的效果
gradient_accumulate_every = 2

# 指数移动平均（EMA）的衰减率
# 用于稳定模型参数，防止模型在训练过程中出现剧烈变化。EMA会生成一个平滑的模型，使用设定的衰减率来控制变化幅度
# 较高的衰减率（0.995）表示模型参数变化相对较小，EMA值主要由过去的模型参数决定
ema_decay = 0.995

# 采样频率
# 每训练多少步进行一次采样，用于生成样本数据以观察训练的效果。采样可以帮助监控模型在训练中的生成能力
sample_freq = 2000

# 保存频率
# 每训练多少步保存一次模型的权重。这是为了在训练中断时，可以从最近的保存点继续训练
# 同时，定期保存也便于后期选择最优的模型参数
save_freq = 2000

# 总训练步数
# 决定整个训练过程的总步数。步数越高，模型越有可能达到良好的拟合效果，但需要耗费更多时间
n_train_steps = int(1e5)  # 1e5为科学记数法，表示100000

# 模型保存的次数
# 设置在整个训练过程中，模型参数被保存的次数。可以控制每隔固定步数进行保存，得到不同阶段的模型
n_saves = 5

# 是否并行保存
# 指定是否在多个进程上同时保存模型参数。在并行训练时可能有用，但这里设置为False表示不使用并行保存
save_parallel = False

# 存储桶
# 在云端保存训练文件时，可以指定存储桶（Bucket），便于分布式存储和读取
# 这里没有使用云端存储，所以设置为 None
bucket = None

# 参考样本数量
# 指定在训练过程中生成的参考样本数量。通过查看这些参考样本，可以观察模型生成的质量
# 设置为 8，即每次采样生成 8 个样本
n_reference = 8

# 批次大小
# 每次训练所使用的数据样本数，决定了每次参数更新所依赖的数据量
# 较大的批次有助于训练稳定，但消耗更多的内存资源。这里设置为 32
train_batch_size = 32

# -------------------------------
# 创建训练器配置
# -------------------------------

# 使用 dConfig 类创建训练器的配置对象
# 训练器配置对象主要用于设定训练过程中各个参数，包括学习率、批次大小、保存路径等

trainer_config = dConfig(
    dTrainer,                         # 指定训练器类，这里使用 dTrainer 作为模型的训练工具
    savepath=(savepath, 'trainer_config.pkl'),  # 配置文件的保存路径，以供后续查看或加载
    train_batch_size=train_batch_size,           # 批次大小
    train_lr=learning_rate,                      # 学习率
    gradient_accumulate_every=gradient_accumulate_every,  # 梯度累积的步数
    ema_decay=ema_decay,                         # 指数移动平均的衰减率，用于平滑训练参数
    sample_freq=sample_freq,                     # 采样频率，每隔多少步采样一次
    save_freq=save_freq,                         # 保存频率，每隔多少步保存一次模型
    label_freq=int(n_train_steps // n_saves),    # 标签更新频率，每隔指定步数输出标签，用于查看保存情况
    save_parallel=save_parallel,                 # 是否并行保存模型，设置为 False
    results_folder=savepath,                     # 结果文件的保存路径
    bucket=bucket,                               # 存储桶，用于云端存储，设置为 None
    n_reference=n_reference                      # 参考样本数量，每次采样生成的参考样本数
)

# -------------------------------
# 实例化训练器
# -------------------------------

# 使用配置对象创建训练器实例
# 传入扩散模型（diffusion）和数据集（dataset）作为训练器的输入，renderer 设为 None，因为不进行可视化渲染
trainer = trainer_config(diffusion, dataset, renderer=None)



[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /Users/xieyongyan/Desktop/third year 项目/Third_year/DeepMimic_diffusion_mujoco/diffusion/logs/test-cartwheel-shuffled-128
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /Users/xieyongyan/Desktop/third year 项目/Third_year/DeepMimic_diffusion_mujoco/diffusion/logs/test-cartwheel-shuffled-128/trainer_config.pkl



### 6. 训练模型 （测试）

In [8]:
# 测试训练模型

import torch
from diffuser.utils import batchify

# 将模型移动到指定设备
tunet = model
tunet.to(device)

# 获取一个测试数据样本
test_data = dataset[0]

# 将数据样本转换为批次形式
batch = batchify(test_data)

# 随机生成一个时间步 t
t = torch.randint(0, n_timesteps, (1,), device=device).long()

# 前向传播测试
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)

# 输出结果和形状
print(res.shape)


x.shape torch.Size([1, 160, 69])
x.shape torch.Size([1, 69, 160])
x.shape torch.Size([1, 128, 160])
1
xfinal.shape torch.Size([1, 128, 80])
x.shape torch.Size([1, 256, 80])
2
xfinal.shape torch.Size([1, 256, 40])
x.shape torch.Size([1, 512, 40])
3
xfinal.shape torch.Size([1, 512, 20])
x.shape torch.Size([1, 1024, 20])
4
xfinal.shape torch.Size([1, 1024, 20])
xt1.shape torch.Size([1, 1024, 20])
xt2.shape torch.Size([1, 1024, 20])
xt3.shape torch.Size([1, 1024, 20])
pop.shape torch.Size([1, 1024, 20])
pop.shape torch.Size([1, 512, 40])
pop.shape torch.Size([1, 256, 80])
torch.Size([1, 160, 69])


In [9]:
# 测试损失计算和反向传播

from diffuser.utils import report_parameters, batchify

# 报告模型的参数数量
report_parameters(model)

# 测试损失计算和反向传播
print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')


[ utils/arrays ] Total parameters: 63.13 M
         downs.3.0.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 5.24 M | Conv1d(2048, 512, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### 6. 训练模型

In [ ]:
# 计算每个 epoch 的训练步数和总的 epoch 数
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(f"Total epochs: {n_epochs}")

# 为了示例，将 epoch 数设置为 3
n_epochs = 2

# 开始训练循环
for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs} | Savepath: {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

# 保存模型
trainer.save(n_epochs)


Total epochs: 100
Epoch 1/2 | Savepath: /Users/xieyongyan/Desktop/third year 项目/Third_year/DeepMimic_diffusion_mujoco/diffusion/logs/test-cartwheel-shuffled-128
[ utils/training ] Saved model to /Users/xieyongyan/Desktop/third year 项目/Third_year/DeepMimic_diffusion_mujoco/diffusion/logs/test-cartwheel-shuffled-128/state_0.pt
0:   0.8667 | a0_loss:   0.1125 | t:   9.4874
100:   0.1989 | a0_loss:   0.0191 | t: 849.6004
200:   0.1007 | a0_loss:   0.0090 | t: 855.9119


### （6. 加载模型)

In [ ]:
# 加载指定 epoch 的模型检查点
trainer.load(1)

# 使用训练器中的 EMA 模型进行推理或采样
model = trainer.ema_model


### 7. 从模型中采样

In [ ]:
from diffuser.utils import batchify

# 获取一个测试数据样本并转换为批次形式
test = dataset[0]
batch = batchify(test)

# 使用扩散模型生成采样的轨迹
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions)


### 8. 保存生成的运动数据


In [ ]:
import numpy as np

def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    
    # 提取采样轨迹的前 35 个维度（位置数据）
    pos_data = sample.trajectories[:, :, :pos_dim]
    print(f"Sampled motion shape: {pos_data.shape}")
    
    # 去除批次维度，并将数据移动到 CPU
    pos_data = pos_data.squeeze(0).cpu().numpy()
    
    # 保存位置数据到指定的 .npy 文件
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")

# 调用函数，将采样的运动数据保存到指定目录
save_motions(sample, os.path.join(savepath, "sampled_motions"), filename="base-motion2.npy")
